# **Download do Dataset**

In [ ]:
!kaggle datasets download -d jessicali9530/lfw-dataset -p ../data/

In [ ]:
!unzip -q ../data/lfw-dataset.zip -d ../data/

In [ ]:
!rm ../data/lfw-dataset.zip
!rm ../data/*.csv

# **Detecção das faces**

In [1]:
from facenet_pytorch import MTCNN, extract_face, InceptionResnetV1
import torch
from torchvision.transforms import Compose, Resize, ToTensor, Normalize, Lambda
from PIL import Image
import os
from tqdm.notebook import tqdm as tqdm
import pandas as pd
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device name: {torch.cuda.get_device_name(0)}')

transform = Compose(
    [
    Resize((160, 160)), 
    ToTensor(), 
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
)

RAW_LFW_PATH = '../data/lfw-deepfunneled/lfw-deepfunneled/'
FACES_PATH = '../data/lfw-faces/'

Device name: NVIDIA GeForce RTX 3050 Laptop GPU


In [ ]:
mtcnn = MTCNN(keep_all=True, device=device)

In [ ]:
def extract_faces(raw_images_path: str, faces_images_path: str) -> pd.DataFrame:
    if not os.path.exists(faces_images_path):
        os.makedirs(faces_images_path)
        
    subfolders = os.listdir(raw_images_path)
    
    nobox = []
    df = []
    
    for id, folder in enumerate(tqdm(subfolders, desc='Extraindo rostos', unit='img')):
        imgs = os.listdir(f'{raw_images_path}/{folder}')
        
        for file in imgs:
            save_path = os.path.join(faces_images_path, file.split('/')[-1])
            save_path_df = os.path.join('./data/lfw-faces/', file.split('/')[-1])
            df.append((id, save_path_df))
            
            img = Image.open(os.path.join(raw_images_path + folder, file))
        
            boxes, _ = mtcnn.detect(img)
            
            if boxes is not None:
                extract_face(img, boxes[0], save_path=save_path)
            else:
                nobox.append(save_path_df)
                
    df = pd.DataFrame(df, columns=['id', 'path'])
    
    return nobox, df

In [ ]:
nobox, df = extract_faces(RAW_LFW_PATH, FACES_PATH)

print(f'Número de imagens sem face detectada: {len(nobox)}')
print(f'Número de imagens com face detectada: {len(df)}')
print(f'Número de identidades: {df["id"].nunique()}')

# Removendo imagens sem face detectada do df
df = df[~df['path'].isin(nobox)]

# Salvando o csv
df.to_csv('../data/lfw_faces.csv', index=False)

Extraindo rostos:   0%|          | 0/5749 [00:00<?, ?img/s]

Número de imagens sem face detectada: 1
Número de imagens com face detectada: 13233
Número de identidades: 5749


In [ ]:
df.head()

,id,path
0,0,./data/lfw-faces/Koichiro_Matsuura_0001.jpg
1,1,./data/lfw-faces/Mark_Hanson_0001.jpg
2,2,./data/lfw-faces/Gregorio_Honasan_0001.jpg
3,3,./data/lfw-faces/Shanna_Zolman_0001.jpg
4,4,./data/lfw-faces/Edward_Seaga_0001.jpg


# **Split de treino e teste**

In [ ]:
!rm -rf ../data/lfw-deepfunneled/

In [ ]:
ids_count = df['id'].value_counts()
valid_ids = ids_count[ids_count >= 5].index

shuffled_ids = valid_ids.to_numpy()
np.random.seed(42)
np.random.shuffle(shuffled_ids)

test_ids = shuffled_ids[:49]

# Criar os dataframes de treino e teste
test_df = df[df['id'].isin(test_ids)]
train_df = df[~df['id'].isin(test_ids)]

print(f"Identidades no conjunto de teste: {test_df['id'].nunique()}")
print(f"Imagens no conjunto de teste: {len(test_df)}\n")

print(f"Identidades no conjunto de treino: {train_df['id'].nunique()}")
print(f"Imagens no conjunto de treino: {len(train_df)}")

Identidades no conjunto de teste: 49
Imagens no conjunto de teste: 567

Identidades no conjunto de treino: 5700
Imagens no conjunto de treino: 12665


# **Extração dos embeddings**

In [ ]:
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

In [ ]:
def extract_embeddings(model, train_df, transform, device) -> list:
    embeddings = []
    for i, row in tqdm(train_df.iterrows(), total=len(train_df)):
        img = Image.open(row['path'])
        img = transform(img).to(device)
        img_embedding = model(img.unsqueeze(0)).detach().cpu().numpy()
        embeddings.append(img_embedding)
    
    return embeddings

In [ ]:
embeddings = extract_embeddings(resnet, train_df, transform, device)
train_df['embedding'] = embeddings

In [12]:
train_df.to_pickle('../data/lfw_train_embeddings.pkl')
test_df.to_csv('../data/lfw_test.csv', index=False)